In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
from zipfile import ZipFile
import zipfile
import fnmatch
import pandas as pd
import tqdm
import time
import re
import numpy as np
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df = pd.read_csv('/content/songs_features')

In [4]:
# Фичи, полученные при помощи emovecAPI
emovec = pd.read_csv('/content/emotional_vectors.csv')

### Preprocessing

обрабатываю основной датасет с признаками

In [5]:
def genre_preprocess(df):
    df['genres'] = df['genres'].apply(lambda x: x.split(", "))
    df['genres'] = df['genres'].apply(lambda x: [elem.strip('_') for elem in x])
    tfidf = TfidfVectorizer()
    tfidf_matrix =  tfidf.fit_transform(df['genres'].apply(lambda x: " ".join(x)))
    genre_df = pd.DataFrame(tfidf_matrix.toarray())
    genre_df.columns = [i for i in tfidf.get_feature_names_out()]
    genre_df.drop(columns='unknown')
    genre_df.reset_index(drop = True, inplace = True)
    genre_df.drop(columns=['_hip_hop'], inplace=True)
    return genre_df

genre_df = genre_preprocess(df)

In [6]:
def get_sentiment_analisys(df):
  df['subjectivity'] = df['track_name'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
  df['polarity'] = df['track_name'].apply(lambda x: TextBlob(x).sentiment.polarity)
  return df 

df = get_sentiment_analisys(df)

In [7]:
df.drop(columns=['genres'], inplace=True)

In [8]:
def normalize(df):
  float_df = df[['acousticness', 'danceability', 'year', "artist_popularity", "song_popularity",
       'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
       'tempo', 'valence', 'key', 'mode', 'subjectivity', 'polarity']]
  scaler = MinMaxScaler()
  scaled_float_df = pd.DataFrame(scaler.fit_transform(float_df), columns = float_df.columns) * 0.5
  return scaled_float_df

scaled_float_df = normalize(df)

In [9]:
features_df = pd.concat([df[['id']], genre_df, scaled_float_df], axis = 1) 

In [10]:
features_df.head(3)

,id,432hz,_indie,a_cappella,abstract,acoustic_blues,acoustic_pop,adult_standards,afro_psych,afrobeat,...,instrumentalness,liveness,loudness,speechiness,tempo,valence,key,mode,subjectivity,polarity
0,spotify:track:5IbCV9Icebx8rR6wAp5hhP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.069744,0.363330,0.042089,0.190915,0.327919,0.090909,0.5,0.00,0.25
1,spotify:track:6rKVAvjHcxAzZ1BHtwh5yC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.086667,0.348567,0.027874,0.341100,0.458883,0.000000,0.5,0.00,0.25
2,spotify:track:6Jlkb1Wh08RYHstWScsTvg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000005,0.052308,0.311609,0.023980,0.188055,0.160406,0.272727,0.5,0.05,0.25


### Отбор 50 плейлистов из миллиона (просто беру первые 50) затем оставляю только те, где треков >= 15, остается 40 плейлистов

In [11]:
with ZipFile('/content/gdrive/MyDrive/spotify_million_playlist_dataset.zip') as zipfiles:
  file_list = zipfiles.namelist()
  json_files = sorted(fnmatch.filter(file_list, "spotify_million_playlist_dataset/data/*.json"), key=lambda x: x[x.index('mpd.slice.'):x.index('.json')])[:2]
  for file in json_files:
    json_file = zipfiles.extract(file)
    print(json_file)

/content/spotify_million_playlist_dataset/data/mpd.slice.0-999.json
/content/spotify_million_playlist_dataset/data/mpd.slice.1000-1999.json


In [12]:
songs = list(emovec['song_name'].values)
playlists = {}
f = open('/content/spotify_million_playlist_dataset/data/mpd.slice.1000-1999.json')
js = f.read()
f.close()
current_slice = json.loads(js)
for playlist in current_slice['playlists'][:50]:
  tracks = []
  for track in playlist['tracks']:
    if track['track_uri'].split(':')[2] in songs:
      tracks.append(track['track_uri'].split(':')[2])
  if len(tracks) >= 15:
    playlists[playlist['name']] = tracks

### Рекомендательная ситема

In [15]:
keys = list(playlists.keys())

In [16]:
def get_result(df):
  array = []
  for key in keys:
    playlist = playlists[key][:5]
    n = len(playlists[key])
    vector = df[df['id'].isin(playlist)].drop(columns=['id']).mean(axis=0)
    all_songs = df[~df['id'].isin(playlist)].drop(columns=['id'])
    try:
      res = cosine_similarity(all_songs.values, vector.values.reshape(1, -1))
    except ValueError:
      continue
    answer = list(pd.DataFrame(res, index=df[~df['id'].isin(playlist)]['id'])[0].nlargest(10).index)
    answer = set(answer)
    reall = playlists[key][5:]
    reall = set(reall)
    try:
      array.append((len(answer & reall)/(10)) * 100)
    except ZeroDivisionError:
      continue
  return np.mean(array)

In [17]:
# первоначальный датасет
get_result(usual_feature_df)

33.75

In [18]:
# с векторами arousal и valence 
get_result(emovec_feature_df)

35.25